https://github.com/avidale/encodechka - сравнение моделей получение эмбеддингов текста относительно задачи семантического сходства текстов

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 127.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.0 MB/s eta 0:00:00


In [2]:
!pip install emoji==1.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=ce3b69d3ad30ede4fa3fbd8ed88b92532b17eae11649023ff31769a623adb3e1
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [3]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from bs4 import BeautifulSoup
import torch
import numpy as np
import re
import emoji

In [4]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [5]:
def preprocess_text(text: str) -> str:
  soup = BeautifulSoup(text, features="html.parser")
  text = soup.get_text()
  text = text.replace("\n", "")
  return text

In [6]:
def remove_url(text: str) -> str:
  return re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', ' ', text)

##  data

In [7]:
df_post = pd.read_csv("post10k.csv", encoding="utf-8")
df_post.head(5)

,Unnamed: 0,msg_id,entity_id,text,content_type,keyboard,silent,has_media_spoiler,posted
0,0,5301,1002094480,🤤 Яркие фотокарточки первой победы в году уже ...,media_group,NaN,0,0,2023-01-09 17:10:06
1,1,5302,1002094480,💥 Кто стал обладателем игровой майки и брендир...,text,NaN,0,0,2023-01-10 14:00:11
2,2,5304,1002094480,🤫💪🏼Тренируемся в традиционном режиме \n\nГотов...,video,NaN,0,0,2023-01-10 14:40:25
3,3,5305,1002094480,<b>Проведи год вместе с Динамо-Ак Барс </b>📆💙\...,video,NaN,0,0,2023-01-11 12:30:23
4,4,5306,1002094480,<b>Динамо-Ак Барс 🆚 Минчанка\n\n</b>Кто<b> </b...,photo,NaN,0,0,2023-01-11 16:00:06


In [8]:
df_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11439 entries, 0 to 11438
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         11439 non-null  int64 
 1   msg_id             11439 non-null  int64 
 2   entity_id          11439 non-null  int64 
 3   text               11083 non-null  object
 4   content_type       11439 non-null  object
 5   keyboard           94 non-null     object
 6   silent             11439 non-null  int64 
 7   has_media_spoiler  11439 non-null  int64 
 8   posted             11439 non-null  object
dtypes: int64(5), object(4)
memory usage: 804.4+ KB


In [9]:
df_post = df_post.dropna(subset=['text'])

In [10]:
df_post = df_post.reset_index(drop=True)



In [11]:
df_post.text = df_post.text.apply(lambda x: preprocess_text(x))
df_post.text = df_post.text.apply(lambda x: remove_url(x))
df_post['text'] = df_post["text"].apply(lambda x: emoji.replace_emoji(x))

<ipython-input-5-f2e004724d36>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, features="html.parser")
<ipython-input-5-f2e004724d36>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, features="html.parser")


## intfloat/multilingual-e5-large

In [12]:
tokenizer_e5 = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model_e5 = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [13]:
def e5_sts(sent1: str, sent2: str) -> float:
    # Tokenize the input texts
    batch_dict = tokenizer_e5([sent1, sent2], max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model_e5(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    scores = (embeddings @ embeddings.T) * 100
    scores = scores.tolist()
    scores = np.divide(scores, 20)
    return scores[0][1]

In [14]:
# Tokenize the input texts
batch_dict = tokenizer_e5([df_post.text[223], df_post.text[0]], max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model_e5(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

In [15]:
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings @ embeddings.T) * 100
scores = scores.tolist()

In [16]:
scores = np.divide(scores, 20)

In [17]:
scores

array([[4.99999924, 3.66301079],
       [3.66301079, 5.00000076]])

## sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [18]:
sentences = [df_post.text[223], df_post.text[0]]

In [19]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [20]:
# Load model from HuggingFace Hub
tokenizer_mpnet = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model_mpnet = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [21]:
def mpnet_sts(sent1: str, sent2: str) -> float:
    sentences = [sent1, sent2]
    # Tokenize sentences
    encoded_input = tokenizer_mpnet(sentences, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model_mpnet(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # Косинусное сходство
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    scores = (sentence_embeddings @ sentence_embeddings.T) * 100
    scores = scores.tolist()
    scores = np.divide(scores, 20)
    return scores[0][1]

In [22]:
# Tokenize sentences
encoded_input = tokenizer_mpnet(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model_mpnet(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [23]:
# Косинусное сходство
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
scores = (sentence_embeddings @ sentence_embeddings.T) * 100
print(scores.tolist())

[[100.00001525878906, 26.426273345947266], [26.426273345947266, 100.0]]


## navec


In [24]:
!pip install navec

In [25]:
!pip install razdel

In [26]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=77801a7dc69b8a5746de71a3c0f736bd8928e4705b5f5159a47377cb0673c616
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [27]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-10-11 08:09:33--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.1s    

2023-10-11 08:09:37 (16.2 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [28]:
from navec import Navec
from razdel import tokenize
import pymorphy2


morph = pymorphy2.MorphAnalyzer()

In [29]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [30]:
embedding_dim = 300

In [31]:
def tokenizer_razdel(text):
    navec_words = []
    tokens = [_.text for _ in list(tokenize(text))]

    for word in [morph.parse(_)[0].normal_form for _ in tokens]:
        try:
            navec[word]
            navec_words.append(word)
        except:
            pass
    return navec_words


In [32]:
def get_sentence_embedding(sentence):
    words = tokenizer_razdel(sentence)
    sentence_len = len(words)
    embedding = np.zeros(embedding_dim)
    for word in words:
        navec[word.lower()]
        embedding += navec[word.lower()]

    return embedding / sentence_len

In [33]:
navec_sentence_embeddings = []
navec_sentence_embeddings.append(get_sentence_embedding(sentences[0]))
navec_sentence_embeddings.append(get_sentence_embedding(sentences[1]))

In [34]:
navec_sentence_embeddings = torch.Tensor(np.array(navec_sentence_embeddings))

In [35]:
# Косинусное сходство
navec_sentence_embeddings = F.normalize(navec_sentence_embeddings, p=2, dim=1)
scores = (navec_sentence_embeddings @ navec_sentence_embeddings.T) * 100
print(scores.tolist())

[[100.0, 55.540443420410156], [55.540443420410156, 100.00001525878906]]


In [36]:
def navec_avg_sts(sent1: str, sent2: str) -> float:
    navec_sentence_embeddings = [get_sentence_embedding(sent) for sent in [sent1, sent2]]
    navec_sentence_embeddings = torch.Tensor(np.array(navec_sentence_embeddings))
    # Косинусное сходство
    navec_sentence_embeddings = F.normalize(navec_sentence_embeddings, p=2, dim=1)
    scores = (navec_sentence_embeddings @ navec_sentence_embeddings.T) * 100
    scores = scores.tolist()
    scores = np.divide(scores, 20)
    return scores[0][1]

## load test data

In [37]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [38]:
from datasets import load_dataset

In [39]:
dataset = load_dataset("stsb_multi_mt", name="ru", split="dev")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [40]:
df = pd.DataFrame(dataset)

In [41]:
df

,sentence1,sentence2,similarity_score
0,Человек в твердой шляпе танцует.,Мужчина в твердой шляпе танцует.,5.00
1,Маленький ребенок едет верхом на лошади.,Ребенок едет на лошади.,4.75
2,Мужчина кормит мышь змее.,Человек кормит змею мышью.,5.00
3,Женщина играет на гитаре.,Человек играет на гитаре.,2.40
4,Женщина играет на флейте.,Человек играет на флейте.,2.75
...,...,...,...
1495,"Ученые доказали, что на Марсе есть вода.",Наса обнаружила воду на Марсе?,2.00
1496,"Пранаб подчеркивает, что государства должны ст...",ВТО: Индия сожалеет о действиях развитых стран,0.00
1497,Фольксваген заносит красный цвет после скандал...,"Жест доброй воли"" Фольксвагена в адрес владель...",2.00
1498,Обама прав: Африка заслуживает лучшего лидерства,"Обама ждет середины семестра, чтобы назвать им...",0.00


## Testing

In [42]:
def mae(dataframe, func, column_name):
    maes = [np.nan] * len(dataframe)
    for s in range(len(dataframe)):
        ent = dataframe.iloc[s]
        maes[s] = abs(func(ent.sentence1, ent.sentence2) - ent.similarity_score)

    dataframe[column_name] = maes

In [43]:
df1 = df.copy()

In [44]:
df1 = df1.iloc[lambda x: x.index % 5 == 0]

In [45]:
df1.iloc[41]

sentence1                        Мужчина подает торт.
sentence2           Мужчина, приправляющий перепелов.
similarity_score                                 0.75
Name: 205, dtype: object

In [46]:
for f, colname in ((e5_sts, "e5_mae"), (mpnet_sts, "mpnet_mae"), (navec_avg_sts, "navec_mean_mae")):
    mae(df1, f, colname)

In [47]:
df1

,sentence1,sentence2,similarity_score,e5_mae,mpnet_mae,navec_mean_mae
0,Человек в твердой шляпе танцует.,Мужчина в твердой шляпе танцует.,5.000,0.021396,0.062128,0.271412
5,Женщина режет лук.,Мужчина режет лук.,2.615,2.069937,0.946666,2.029320
10,Люди играют в крикет.,Мужчины играют в крикет.,3.200,1.654596,1.406905,1.494329
15,Человек что-то находит.,Женщина что-то режет.,0.800,3.241681,1.165920,2.428532
20,Женщина готовит яйца.,Женщина что-то готовит.,3.000,1.619082,0.680216,0.992305
...,...,...,...,...,...,...
1475,Президент Алжира получает терапию после инсуль...,Президент Болгарии пытается выйти из предвыбор...,0.400,3.684409,0.769936,1.784677
1480,Предложение Казначейства предусматривает увели...,Казначейство предлагает повысить подоходный на...,4.000,0.839022,0.751891,0.162353
1485,Мальдивы начинают голосовать в контролируемом ...,Ударившиеся в кризис Мальдивы провели новый пр...,3.000,1.397646,0.640503,0.105276
1490,Назначен новый руководитель миротворческой мис...,ООН берет на себя ответственность за поддержан...,2.000,2.555613,1.706604,0.988348


In [48]:
df1.describe()

,similarity_score,e5_mae,mpnet_mae,navec_mean_mae
count,300.000000,300.000000,300.000000,300.000000
mean,2.329728,2.051060,0.868256,1.378725
std,1.483120,1.196999,0.610516,0.986827
min,0.000000,0.000000,0.000001,0.000000
25%,0.950000,1.093629,0.398373,0.586351
50%,2.600000,1.867516,0.742427,1.145747
75%,3.500000,3.145647,1.225935,2.064859
max,5.000000,4.131989,2.869203,4.141666


In [49]:
df1.to_csv('result.csv', sep=',')

In [50]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

<Axes: >

In [51]:
df1[["mpnet_mae", "e5_mae", "navec_mean_mae"]].plot(figsize=(20,3))